In [2]:
import pandas as pd
import glob

path = '../data/raw/*.csv'
csv_files = glob.glob(path)
df = []

for file in csv_files:
    df = pd.read_csv(file)
    print(list(df))

['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']


In [3]:
def shift_column(df, col):
    col_list = list(df)
    #col1_index = df.columns.get_loc(col1) #or list(df.columns).index(col1)
    #col2_index = df.columns.get_loc(col2)
    #col_list[col1_index], col_list[col2_index] = col_list[col2_index], col_list[col1_index]
    #df.columns = col_list
    col1 = df.pop(col)
    df.insert(len(col_list) - 1, col, col1)

for file in csv_files:
    df1 = pd.read_csv(file)
    if 'remaining_lease' in df1.columns:
        shift_column(df1, 'remaining_lease') #idt need to define a separate function
        print(df1[:5])
    df = pd.concat([df, df1]) #I realised I forgot to exclude header when concatenating...

print(df[:5])

     month        town flat_type block        street_name storey_range  \
0  2017-01  ANG MO KIO    2 ROOM   406  ANG MO KIO AVE 10     10 TO 12   
1  2017-01  ANG MO KIO    3 ROOM   108   ANG MO KIO AVE 4     01 TO 03   
2  2017-01  ANG MO KIO    3 ROOM   602   ANG MO KIO AVE 5     01 TO 03   
3  2017-01  ANG MO KIO    3 ROOM   465  ANG MO KIO AVE 10     04 TO 06   
4  2017-01  ANG MO KIO    3 ROOM   601   ANG MO KIO AVE 5     01 TO 03   

   floor_area_sqm      flat_model  lease_commence_date  resale_price  \
0            44.0        Improved                 1979      232000.0   
1            67.0  New Generation                 1978      250000.0   
2            67.0  New Generation                 1980      262000.0   
3            68.0  New Generation                 1980      265000.0   
4            67.0  New Generation                 1980      265000.0   

      remaining_lease  
0  61 years 04 months  
1  60 years 07 months  
2  62 years 05 months  
3   62 years 01 month  
4 

In [4]:
print(len(df))
df.drop_duplicates()
print(len(df))

1228461
1228461


In [5]:
df.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
dtype: object

In [6]:
for date in df['month']:
    date += '-01'
    pd.to_datetime(date) #too slow, took 621s to execute

df.sort_values(by='month')

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
498,1990-01,BUKIT MERAH,5 ROOM,7,KIM TIAN PL,16 TO 18,131.0,MODEL A,1984,180000.0,NaN
499,1990-01,BUKIT MERAH,5 ROOM,115,BT PURMEI RD,04 TO 06,135.0,MODEL A,1985,180000.0,NaN
500,1990-01,BUKIT MERAH,5 ROOM,48,TELOK BLANGAH DR,04 TO 06,123.0,STANDARD,1976,139000.0,NaN
501,1990-01,BUKIT MERAH,5 ROOM,73,TELOK BLANGAH HTS,07 TO 09,117.0,STANDARD,1978,196000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
194958,2024-12,CHOA CHU KANG,4 ROOM,803D,KEAT HONG CL,04 TO 06,93.0,Model A,2017,590000.0,91 years 07 months
194959,2024-12,CLEMENTI,4 ROOM,512,WEST COAST DR,04 TO 06,91.0,New Generation,1978,488000.0,52 years 03 months
194960,2024-12,GEYLANG,2 ROOM,109,ALJUNIED CRES,04 TO 06,45.0,Improved,1986,327300.0,60 years 03 months
194982,2024-12,JURONG WEST,5 ROOM,624,JURONG WEST ST 61,07 TO 09,110.0,Improved,2001,610000.0,75 years 04 months


In [9]:
df.fillna(0, inplace=True)

def numbers_only(string):
    year_index = string.index(' y')
    year = string[:year_index]
    duration = 0
    if ' months' in string:
        month_index_start = string.index('s ') + 2
        month_index_end = string.index(' m')
        month = string[month_index_start:month_index_end]
        month_adjusted = str(float(month) / 12)
        duration = float(year + month_adjusted)
    else:
        duration = year
    return duration 

try:
    df['remaining_lease'].astype('int32')
except ValueError:
    df['remaining_lease'].apply(lambda x: numbers_only(x) if type(x) == str else None) 

In [80]:
# do I need to check for + remove outliers?
# do I need to standardise/normalise data?

In [37]:
for column in df.columns:
    exclude = ['month', 'floor_area_sqm', 'lease_commence_date', 'resale_price', 'remaining_lease']
    if column not in exclude:
        df[column] = df[column].str.replace('-', ' ').str.lower()
        df[column] = df[column].astype('category')

In [41]:
df.dtypes
for c in df['storey_range'].cat.categories:
    print(c) 
# for storey_range do I combine 01 to 03 and 01 to 05? there's overlap but might lose data
# or maybe I split 01 to 03 into 01 02 03?

01 to 03
01 to 05
04 to 06
06 to 10
07 to 09
10 to 12
11 to 15
13 to 15
16 to 18
16 to 20
19 to 21
21 to 25
22 to 24
25 to 27
26 to 30
28 to 30
31 to 33
31 to 35
34 to 36
36 to 40
37 to 39
40 to 42
43 to 45
46 to 48
49 to 51
